# Pipeline analysis for supervised data
Let's start simple by analyzing the data from a single individual, considering: total distance, look-around, sniffing, inner-zone time, wall-climbing, and huddle. We start by importing the necessary packages and directories.

In [ ]:
import os
import pandas as pd
import pickle
import deepof.data
import deepof.visuals
import visuals_customized
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Open an existing DeepOF project
directory_output = '//folder/becell/Lab Projects/ERCstG_HighMemory/Data/Marc/1) SOC/2024-01a02 cFos/'
my_deepof_project = deepof.data.load_project(directory_output + "deepof_tutorial_project")

In [ ]:
# Open an existing supervised analysis
with open(directory_output + 'supervised_annotation.pkl', 'rb') as file:
    supervised_annotation = pickle.load(file)

In [ ]:
# Load conditions
my_deepof_project.load_exp_conditions(directory_output + 'conditions.csv')

In [ ]:
# Define colors
white = '#FFFFFF'
grey_soft = '#D3D3D3'
grey_stark = '#636466'
blue = '#194680'
red = '#801946'

## Plot PCA between conditions

In [ ]:
# Define the coordinates for the labels of each condition
coords_dict = {
    'paired': ['paired', [1,3]],
    'unpaired': ['unpaired', [2,4]],
    'noshock': ['no-shock', [3,5]],
    }

In [ ]:
# Define the colors for each condition
color_dict = {
    'paired': grey_stark,
    'unpaired': blue,
    'noshock': red
    }

In [ ]:
ax, embedding_dataset, dataframe_for_titles, pca = visuals_customized.plot_embeddings(
    my_deepof_project,
    supervised_annotations=supervised_annotation,
    bin_size=60,
    bin_index=2,
    my_title = '',
    my_color_dict = color_dict,
    my_coords_dict = coords_dict,
    # given_pca = pca # Uncomment to train a new pca
)

### Analyze individual PC and rotated_loading_scores

In [ ]:
# Perform statistics
import pingouin as pg
pg.normality(embedding_dataset, 'PCA-2', 'experimental condition')

In [ ]:
pg.homoscedasticity(embedding_dataset, 'PCA-2', 'experimental condition')

In [ ]:
x = embedding_dataset[embedding_dataset['experimental condition'] == 'hc_ee']['PCA-2']
y = embedding_dataset[embedding_dataset['experimental condition'] == 'hc_ind']['PCA-2']
pg.ttest(x, y).round(3)

In [ ]:
# pg.anova(embedding_dataset, 'PCA-1', 'experimental condition')
# pg.pairwise_tukey(embedding_dataset, 'PCA-1', 'experimental condition')

In [ ]:
# Save your statistics in a dictionary
stats_dict = {
    '**': ['paired', 'unpaired']
    }

In [ ]:
# Plot the PC components
visuals_customized.boxplot(embedding_dataset, color_dict, 'PCA-1', stats_dict)

In [ ]:
# Plot the rotated loading scores of PC components
rotated_loading_scores = pca.components_
visuals_customized.lollipop(dataframe_for_titles, rotated_loading_scores, 'PCA-1')

## Plot PCA between time points

In [ ]:
# Define the coordinates for the labels of each condition
coords_dict = {
#    'bin0': ['name1', [4,5]],
#    'bin1': ['name2', [4,5]],
    'bin2': ['Before tone', [-4,4]],
    'bin3': ['During tone', [4,3.5]],
    }

In [ ]:
# Define the colors for each condition
color_dict = {
#    'bin0': sns.color_palette("Set1")[0],
#    'bin1': sns.color_palette("Set1")[1],
    'bin2': grey_stark,
    'bin3': blue
    }

In [ ]:
ax, embedding_dataset, dataframe_for_titles, concat_hue, pca = visuals_customized.plot_embeddings_timelapse(
    my_deepof_project,
    supervised_annotations=supervised_annotation,
    bin_size=60,
    bin_index_list=[2,3],
    my_title = 'Paired',
    my_color_dict = color_dict,
    my_coords_dict = coords_dict,
    specific_condition = 'unpaired',
    given_pca = pca # Uncomment to train a new pca
)

### Analyze individual PC, rotated_loading_scores (across time) 

In [ ]:
# Perform statistics
import pingouin as pg
pg.normality(embedding_dataset, 'PCA-1', 'experimental condition')

In [ ]:
pg.homoscedasticity(embedding_dataset, 'PCA-1', 'experimental condition')

In [ ]:
x = embedding_dataset[embedding_dataset['experimental condition'] == 'bin2']['PCA-1']
y = embedding_dataset[embedding_dataset['experimental condition'] == 'bin3']['PCA-1']
pg.ttest(x, y).round(3)

In [ ]:
pg.anova(embedding_dataset, 'PCA-1', 'experimental condition')
pg.pairwise_tukey(embedding_dataset, 'PCA-1', 'experimental condition')

In [ ]:
# Save your statistics in a dictionary
stats_dict = {
    #'**': ['bin0', 'bin1'],
    #'***': ['bin0', 'bin2'],
    '*': ['bin2', 'bin3'],
    }

In [ ]:
# Plot the PC components
visuals_customized.boxplot(embedding_dataset, color_dict, 'PCA-1', stats_dict)

In [ ]:
# Plot the rotated loading scores of PC components
rotated_loading_scores = pca.components_
visuals_customized.lollipop(dataframe_for_titles, rotated_loading_scores, 'PCA-1')

In [ ]:
# Plot an individual behavior over bins
visuals_customized.timelapse(dataframe_for_titles, concat_hue, 'huddle')

In [ ]:
# Plot some individual behaviors over bins
fig, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(15, 3))

visuals_customized.timelapse(dataframe_for_titles, concat_hue, 'huddle', ax=ax1)
visuals_customized.timelapse(dataframe_for_titles, concat_hue, 'sniffing', ax=ax2)
visuals_customized.timelapse(dataframe_for_titles, concat_hue, 'climbing', ax=ax3)
visuals_customized.timelapse(dataframe_for_titles, concat_hue, 'lookaround', ax=ax4)
visuals_customized.timelapse(dataframe_for_titles, concat_hue, 'speed', ax=ax5)

# ax1.set_title("supervised embeddings of full videos")
# ax2.set_title("supervised embeddings of first two minutes")

plt.tight_layout()
plt.show()